In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ln -s /content/gdrive/My\ Drive/ # drive dosyasına bağlanmak için kod

Mounted at /content/gdrive


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os
import requests
import hashlib

def download_images_from_urls(url_file, output_folder):
    #driver = webdriver.Chrome()  # veya başka bir tarayıcı seçeneği
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with open(url_file, 'r') as file:
        for idx, url in enumerate(file):
            url = url.strip()  # URL'deki boşlukları kaldır

            print(f"Downloading image {idx + 1} from {url}...")
            driver.get(url)
            time.sleep(5)  # Sayfanın tamamen yüklenmesini bekleyin (isteğe bağlı)

            # Resim URL'lerini toplamak için tüm resim elementlerini bul
            image_elements = driver.execute_script("return document.querySelectorAll('img')")

            for img_idx, image_element in enumerate(image_elements):
                # Resmin URL'sini al
                image_url = image_element.get_attribute('src')

                if image_url and image_url.startswith(('http://','https://')):
                    # Resimleri indir
                    response = requests.get(image_url)
                    if response.status_code == 200:
                        # Benzersiz bir dosya adı oluştur
                        unique_filename = hashlib.sha1(image_url.encode()).hexdigest()[:10] + '.jpg' # dosyadaki isimlerin çakışmasını önlemek için
                        with open(os.path.join(output_folder, unique_filename), 'wb') as img_file:
                            img_file.write(response.content)
                            print(f"Image {unique_filename} downloaded successfully.")
                else:
                    print(f"Image {img_idx} URL not valid or not accessible.") # indirilemeyen dosya kontrolü için

    driver.quit()
    print("İşlem tamamlandı.")

url_file = '/content/url.txt'
output_folder = "/content/drive/MyDrive/Colab Notebooks/pool"
download_images_from_urls(url_file, output_folder)


In [ ]:
from google.colab import files

# Google Drive'dan metin dosyasını yükleyin
uploaded = files.upload()

# Yüklenen dosyanın adını alın
file_name = list(uploaded.keys())[0]

# Dosyayı okuyun
with open(file_name, 'r') as file:
    content = file.read()

# İçeriği görüntüleyin
print(content)


In [ ]:
!pip install selenium

#Fazla Alanı Resimden Kaldırma

In [ ]:
import os
import cv2

# Kaynak ve hedef klasörlerin yolları
kaynak_klasor_yolu = "/content/My Drive/Colab Notebooks/empty room"
hedef_klasor_yolu = "/content/My Drive/Colab Notebooks/hedef"

# Hedef klasörün oluşturulması
if not os.path.exists(hedef_klasor_yolu):
    os.makedirs(hedef_klasor_yolu)

# Kaynak klasördeki tüm dosyaları al
dosyalar = os.listdir(kaynak_klasor_yolu)

# Her bir dosya için işlem yap
for dosya in dosyalar:
    # Dosyanın tam yolu
    tam_dosya_yolu = os.path.join(kaynak_klasor_yolu, dosya)

    try:
        # Dosyanın bir resim dosyası olup olmadığını kontrol et
        if os.path.isfile(tam_dosya_yolu) and dosya.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Resmi oku
            resim = cv2.imread(tam_dosya_yolu)

            if resim is not None:
                # Resmin yüksekliğini ve genişliğini al
                yukseklik, genislik = resim.shape[:2]

                # Yükseklikten son 12 pikseli kes
                kesilmis_resim = resim[0:yukseklik-20, :]

                # Dosyanın kaydedileceği hedef yol
                hedef_dosya_yolu = os.path.join(hedef_klasor_yolu, dosya)

                # Kesilmiş resmi hedef klasöre yaz
                cv2.imwrite(hedef_dosya_yolu, kesilmis_resim)
                os.remove(tam_dosya_yolu)
                print("İşlenen dosya ana dizinden silindi")

            else:
                print(f"{dosya} bu dosya okunamadı veya formatı bozuk")

        else:
            print(f"{dosya} resim dosyası yok")

    except Exception as e:
        print(f"{dosya} bu dosyada hata oluştu")

print("Tüm resimler kırpılarak 'hedef_klasor' klasörüne kaydedildi.")
